# Task 1
- 3 regression models

### 1) Linear Regression
- no_of_Mosquito = function(weather_features^A)

### 2) Linear Regression
- no_of_Mosquito_female = function(weather_features^B)
- OR no_of_Mosquito_male = function(weather_features^C)
- Preferably do both

### 3) Polynomial Regression
- no_of_Mosquito_female = function(weather_features^B)
- OR no_of_Mosquito_male = function(weather_features^C)
- Preferably do both

NOTE: A, B, C are a note that a subset of the inputs is used.
Use different cost functions, perform analysis of constructed models, apply normalization and standardization, feature selection

## Dataset

- You need to combine both data sets to prepare a data set suitable for model construction.
- Both data sets have different ‘time grid’ so you need to resolve it.
- You have to aggregate data points over time, and different mosquito species.

### Mosquito data notes:
From website: Capture results of mosquitoes from various locations in Edmonton. These collections are from standard New Jersey light traps that are commonly used to record changes in abundance of mosquitoes before and after control campaigns and to compare seasonal and annual fluctuations in population. Since not all mosquito species are attracted equally to light traps, the City uses a variety of other trapping and survey methods (with their own limitations) to monitor mosquitoes. Not all trap collection sites are factored into the historical averages. Some data can be incomplete due to trap failure. Some trap locations change over time. Trap collections reflect, not absolute population levels, but mosquito activity, which is influenced by changing environmental conditions (temperature, humidity, wind, etc.). The weekly averages do not include any male mosquitoes or any females of species that do not typically bite people. Each data set reflects the mosquito activity of the week previous to the collection date.

To complement this dataset, there is the Rainfall Gauge data which measures rainfall data in the Greater Edmonton area.


- Data collected from May 1990 onward
- Data is updated weekly and collected automatically

#### Columns:

- Trap Date (Date & Time Floating timestamp): date when the mosquito traps are collected. An empty mosquito traps are replaced for the next collection at the same time.
- Genus (Text): When biologists talk about a genus, they mean one or more species of animals or plants that are closely related to each other. Low-level taxonomic ranking for biological classification.
- Specific Epithet (Text): lowest taxonomic rank and having common characteristics and (usually) capable of mating with one another.
- Gender (Text): male or female
- IDd (Text or NaN): specimens too damaged for precise identification or not sigificant in the mosquito program are marked with a value of UnID.
- Count (Number/Int): number of mosquitoes trapped at this Trap Region on this Trap Date.
- Trap Region (Text): The description of where the mosquito trap was placed.
- Include (Text or NaN): blank or no.
- Comparison Group (Text): Trap areas used to evaluate mosquito program efficacy.
- Latitude (Number/Float): geographic coordinate that specifies north-south position of mosquito trap.
- Longitude (Number/Float): The geographic coordinate that specifies the east–west position of the mosquito trap.
- Location (Point): The combination of latitude/longitude for mapping purposes.

## Edmoton weather notes:
- Source: weatherstats.ca based on Environment and Climate Change Canada data

### Columns (personal notes):
- date_time_local (Date/Time human format YYYY-MM-DD HH:MM:SS TZ): date and time data recorded.
- unixtime: unix time format
- pressure_station (float or NaN):
- pressure_sea (float or NaN):
- wind_dir (Text or NaN): wind direction in compass directions (SW, WSW ... etc)
- wind_dir_10s (float or NaN): 
- wind_speed (float or NaN): speed of the wind in CONVUNIT <- TBD
- wind_gust (NaN or float): 
- relative_humidty (float or NaN):
- dew_point (float or NaN): 
- temperature (float or NaN): temp recorded
- windchill (float or NaN):
- humidex (float or NaN):
- visibility (float or NaN):
- health_index (float or NaN):
- cloud_cover_4 (float or NaN):
- cloud_cover_8 (float or NaN):
- cloud_cover_10 (float or NaN):
- solar_radiation (float or NaN):


## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white") #white bg for sns plots
sns.set(style="whitegrid", color_codes=True)

from datetime import timedelta

import time
import warnings
warnings.simplefilter(action='ignore')

## Read csv files

In [ ]:
mos_data = pd.read_csv("./data/mosquito_data.csv")
weather_data = pd.read_csv("./data/weather_data.csv")


orig_mos_data = mos_data.copy()
orig_weather_data = weather_data.copy()

In [ ]:
mos_data.head()

In [ ]:
#dat=mos_data[mos_data["new_trap_date" ]=="2017-06-13"]
#mos_data["new_trap_date"].value_counts()
#mosdat=mos_data.sort_values(by="new_trap_date",inplace=False,ascending=False)
#mergedata=pd.merge(mosdat,weather_data,how='right', on=['new_trap_date', 'new_date_time_local'])

In [ ]:
weather_data.head()

In [ ]:
weather_data.describe()

In [ ]:
mos_data.dtypes

In [ ]:
weather_data.dtypes

In [ ]:
mos_data.isnull().sum()

In [ ]:
weather_data.isnull().sum()

In [ ]:
# drop visibility, cloud_cover_4, cloud_cover_8, cloud_cover_10, solar_radiation as they are all NaN, also humidex and windchill and windgust as they are mostly NaN and nothing that we can do about
weather_data=orig_weather_data.drop(columns=["visibility", "cloud_cover_4", "cloud_cover_8", "cloud_cover_10", "solar_radiation", "wind_gust", "windchill", "humidex"], inplace=False)
# and drop the row with missing values
#weather_data.dropna(inplace=True)
print(weather_data.isnull().sum())
#drop idd include
mos_data=orig_mos_data.drop(columns=["idd","include","geocoded_column","latitude","longitude","geocoded_column"])
#print(mos_data.isnull().sum())
#mos_data.count()
#weather_data.count()

In [ ]:
mos_data["trap_region"].value_counts()

## Date and time 
We can see that both datasets are timestamped but the mosquito dataset is timestamped.
Our first task would be combining both datasets by day and month and year ignoring HH:MM:DD.
- Create a new_trap_date column in mos_data following the format YYYY-MM-DD
- Drop trap_date from mos_data
- create new_date column in weather_data following the format YYYY-MM-DD
- Drop date_time_local and unixtime in weather_data

In [ ]:
mos_data["new_trap_date"] = mos_data.apply(lambda x: x["trap_date"].split("T")[0], axis=1)
#mos_data.drop(columns=["trap_date"], inplace=True) #redundant

mos_data.head()

In [ ]:
#convert new_trap_date into datetime
format = '%Y-%m-%d'
mos_data["new_trap_date"] = pd.to_datetime(mos_data["new_trap_date"], format=format)

mos_data.info()

In [ ]:
#Mountain Daylight Time (MDT) to Mountain Standard Time (MST)
def Tzone(x):
    timelist=x["date_time_local"].split(" ")
    YD=timelist[0]+timelist[1]
    timec=time.mktime(time.strptime(YD,'%Y-%m-%d%H:%M:%S'))
    if timelist[2]=='MDT':
        timec=timec+3600 # add one hour 
    return time.strftime("%Y-%m-%d", time.localtime(timec))
weather_data["new_date_time_local"] = weather_data.apply(lambda x: Tzone(x), axis=1)
weather_data.drop(columns=["date_time_local", "unixtime"], inplace=True)

weather_data["new_date_time_local"] = pd.to_datetime(weather_data["new_date_time_local"], format='%Y-%m-%d')

In [ ]:
weather_data.head()

In [ ]:
# ranges of date/time columns in both dataset
mos_date_range = str(mos_data['new_trap_date'].dt.date.min()) + ' to ' +str(mos_data['new_trap_date'].dt.date.max())
weather_date_range = str(weather_data['new_date_time_local'].dt.date.min()) + ' to ' +str(weather_data['new_date_time_local'].dt.date.max())

print("Range of dates in mosquito dataset is {}".format(mos_date_range))
print("Range of dates in weather dataset is {}".format(weather_date_range))

It can be seen that the mosquito dataset is covering a much bigger range so everything before 2016-12-18 and after 2018-12-18 will be ignored.

In [ ]:
start_remove = weather_data['new_date_time_local'].min()
end_remove = weather_data['new_date_time_local'].max()

In [ ]:
mos_data = mos_data.loc[(mos_data["new_trap_date"] >= start_remove) & (mos_data["new_trap_date"] <= end_remove) ]

mos_date_range = str(mos_data['new_trap_date'].dt.date.min())+' to ' + str(mos_data['new_trap_date'].dt.date.max())
print("Range of dates in mosquito dataset is {}".format(mos_date_range))

In [ ]:
mos_data.sort_values(by="new_trap_date",inplace=True,ascending=True)

In [ ]:
#current distribution:
plt.figure(figsize=(15, 8))
ax = mos_data["count"].hist(bins=500, density=True, stacked=True, color='teal', alpha=0.6)
mos_data["count"].plot(kind='density', color='teal')


ax.set(xlabel="count")
plt.xlim(-10,400)
plt.show()

In [ ]:
mos_data.isnull().sum()

In [ ]:
len(mos_data.index)

### Grouping weather data on date column


In [ ]:
len(weather_data.index)

Replace null data in weather_data

In [ ]:
weather_data.isnull().sum()

In [ ]:
weather_data.reset_index(drop=True, inplace=True)

It looks like health index is normally distributed so its safe to replace na with mean value:

In [ ]:
weather_data.isnull().sum()

In [ ]:
weather_data.head()

In [ ]:
perday_weather_data = weather_data.groupby("new_date_time_local").agg({ #,as_index=False
    #"new_date_time_local": 'first',
    "pressure_station": lambda x: round(x.mean(),2),
    "pressure_sea": lambda x: round(x.mean(),1),
    "wind_dir":lambda x: x.value_counts().index[0],
    "wind_dir_10s":lambda x: x.value_counts().index[0],
    "wind_speed":lambda x: round(x.mean(),1),#x.mode().iloc[0],
    "relative_humidity":lambda x: x.median(),
    "dew_point":lambda x: x.median(),
    "temperature":lambda x: x.median(),
    "health_index": lambda x: round(x.mean(),1),
})
#perday_weather_data.reset_index()
perday_weather_data.head()

#### seperate the wind_dir into 4 columns : wind_dir_W,wind_dir_S,wind_dir_N,wind_dir_E

In [ ]:
wind_dirc=perday_weather_data["wind_dir"].str.split('', expand=True)
wind_dirc

In [ ]:
wind_dir_num=wind_dirc.apply(lambda x:x.value_counts()[1:4],axis = 1).fillna(0)
wind_dir_num.rename(columns={'E':'wind_dir_E', 'N': 'wind_dir_N', 'S': 'wind_dir_S', 'W': 'wind_dir_W'}, inplace=True) 

In [ ]:
perday_weather_data=perday_weather_data.join(wind_dir_num)

In [ ]:
perday_weather_data=perday_weather_data.drop(columns=["wind_dir"])

#### calculate the delta_pressure

In [ ]:
perday_weather_data["delta_pressure"]=perday_weather_data.apply(lambda x:x["pressure_station"]-x["pressure_sea"], axis=1)
perday_weather_data

In [ ]:
mos_data.isnull().sum()

In [ ]:
index_mos_data=mos_data.drop(columns=["trap_date"])
#index_mos_data=mos_data.set_index("new_trap_date")

index_mos_data.head()

In [ ]:
len(perday_weather_data.index)

In [ ]:
#perday_weather_data = perday_weather_data.set_index（'cuspin'，append = True）

## Merge Data

In [ ]:
mergedata=pd.merge(left=perday_weather_data , right=mos_data, left_on="new_date_time_local", right_on="new_trap_date", how='outer', indicator=True)

In [ ]:
mergedata.isnull().sum()
mergedata.describe()

a lot of data of mos_data is null, we need to fill them and because the number of mosquitos would not change sharply, so we can assume the data of mosquitos is same for 5 days

In [ ]:
#group_mos=index_mos_data.groupby("new_trap_date")
group_mo=index_mos_data[index_mos_data["new_trap_date"]=="2017-05-23"]

end_date=group_mo["new_trap_date"].dt.date.min()+timedelta(days=2)

date_rang=pd.DataFrame({"date":pd.date_range(end=pd.to_datetime(end_date),periods=5)})
date_rang=date_rang.groupby("date").apply(lambda df,group=group_mo:group)
date_rang=date_rang.reset_index().drop(columns=["level_1"])
date_rang
#pd.Timestamp(end_date)

In [ ]:
def fill_mos_data(x):
    end_date=x["new_trap_date"].dt.date.min()+timedelta(days=2)
    date_rang=pd.date_range(end=end_date,periods=5)
    date_rang=pd.DataFrame({"date":pd.date_range(end=pd.to_datetime(end_date),periods=5)})
    date_rang=date_rang.groupby("date").apply(lambda df,group_mo=x:group_mo)
    return date_rang.reset_index().drop(columns=["level_1"])
    

In [ ]:
fullday_mos=index_mos_data.groupby("new_trap_date", as_index=False).apply(fill_mos_data)
#fullday_mos=fullday_mos[fullday_mos["trap_region"]=="Rural-West"]

In [ ]:
mergedata=pd.merge(left=perday_weather_data , right=fullday_mos, left_on="new_date_time_local", right_on="date", how='right').set_index("date").reset_index()

In [ ]:
mergedata

We combine nearly all data from weather and mosquito datasets, and there aren't empty values occurring.

## Part A: Linear regression num of Mosquitos

### Select features for Linear regression 

In [ ]:
#sns.pairplot(mergedata)
Lr_data=mergedata.drop(columns=["pressure_sea","genus","specific_epithet","gender","trap_region","comparison_group"])
#Lr_data=Lr_data.loc[Lr_data["count"]<=400]
Lr_data.head()

In [ ]:
# Lr_data=Lr_data.groupby("new_trap_date").agg({
#     'pressure_station':'mean',
#     'delta_pressure':'mean',
#     'wind_dir_10s':'mean',
#     'wind_speed':'mean',
#     'relative_humidity':'mean',
#     'dew_point':'mean',
#     'temperature':'mean',
#     'health_index':'mean',
#     'wind_dir_W':'mean',
#     'count':'median',
#     #'new_trap_date':'first'
# }).reset_index()
#Lr_data=Lr_data[Lr_data["count"]<=Lr_data["count"].describe()["75%"] ]
Lr_data=Lr_data.groupby("date").agg("median").reset_index()
#Lr_data["count"]=Lr_data.groupby("new_trap_date")["count"].transform('sum')

In [ ]:
Lr_data

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
sns.heatmap(Lr_data.corr(), annot=True, ax=ax)

pressure_station	wind_dir_10s	wind_speed	relative_humidity	dew_point	temperature	health_index	

In [ ]:
sns.distplot(Lr_data["count"])

### Training and testing datasets

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import sklearn as sk

train and test data

In [ ]:
Target_data=Lr_data.loc[:,['count']].values
#"delta_pressure","relative_humidity","dew_point"
Feature_data=Lr_data.loc[:,["pressure_station","wind_dir_10s","wind_speed","relative_humidity","dew_point","temperature","health_index"]].values

Feature_data_stand= sk.preprocessing.scale(Feature_data)
Feature_data_norm= sk.preprocessing.minmax_scale(Feature_data)
Feature_data_normalize=sk.preprocessing.normalize(Feature_data)

stand_scaler= sk.preprocessing.StandardScaler()
norm_scaler=sk.preprocessing.MinMaxScaler()
normalize_scaler=sk.preprocessing.Normalizer()
sklreg = linear_model.LinearRegression()
normfeature=norm_scaler.fit_transform(Feature_data)
#Feature_data=Lr_data.drop(columns=["date","count"]).values
X_train, X_test, Y_train, Y_test = train_test_split( Feature_data , Target_data , test_size = 0.3, random_state = 0)

In [ ]:
def cross_val(X,Y,model,scoring_type):
    results=[]
    for score in scoring_type:
        scores = cross_val_score(model, X, Y,
                                     scoring=score, 
                                     cv=10, error_score=np.nan)
        #print(score,":", "mean:", scores.mean(),"std:", scores.std())
        results.append(scores.mean())
    series=pd.Series(results,index=scoring_type)
    return series
        

In [ ]:
Linear=sklreg
Linear_stand = Pipeline([("scaler",stand_scaler),
                     ("linear_regression", sklreg)])
Linear_norm = Pipeline([("scaler",norm_scaler),
                     ("linear_regression", sklreg)])
Linear_normalize = Pipeline([("scaler",normalize_scaler),
                     ("linear_regression", sklreg)])

In [ ]:
results_df = pd.DataFrame()
soclling=["neg_mean_absolute_error","neg_mean_squared_error","neg_root_mean_squared_error","r2"]
series=cross_val(Feature_data,Target_data,Linear,soclling)
series.name="Liner"
results_df=results_df.append(series)
########
series=cross_val(Feature_data,Target_data,Linear_stand,soclling)
series.name="Linear_stand"
results_df=results_df.append(series)
########
series=cross_val(Feature_data,Target_data,Linear_norm,soclling)
series.name="Linear_norm"
results_df=results_df.append(series)
results_df

In [ ]:
sklreg = linear_model.LinearRegression()
sklreg.fit(X_train,Y_train)
np.array([sklreg.intercept_,sklreg.coef_[0]])

In [ ]:
Y_pred_sk = sklreg.predict(X_test)

In [ ]:
plt.scatter(X_test[:,2],Y_test,label='real')
plt.scatter(X_test[:,2],Y_pred_sk, label='pred')
#plt.scatter(Y_test,Y_pred_sk, label='comparison')
plt.legend();

In [ ]:
plt.scatter(X_test[:,0],Y_test,label='real')
plt.scatter(X_test[:,0],Y_pred_sk, label='pred')

In [ ]:
def cross_val(model,x,y):
    pred = cross_val_score(model, x, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
test_pred = sklreg.predict(X_test)
train_pred = sklreg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(Y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(Y_train, train_pred)

In [ ]:
results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(Y_test, test_pred) , cross_val(sklreg,Feature_data,Target_data)]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df

### polynomial models for predicting the number of female or male mosquitos

#### select data

In [ ]:
Po_data=mergedata.drop(columns=["pressure_sea","genus","specific_epithet","trap_region","comparison_group","new_trap_date"])
Po_data.head()

In [ ]:
def count_female_male(x):
    x["Female"]= 0 if x[x["gender"]=="Female"]["count"].sum()==0 else x[x["gender"]=="Female"]["count"].median()
    x["Male"]= 0 if x[x["gender"]=="Male"]["count"].sum()==0 else x[x["gender"]=="Male"]["count"].median()
    return x[0:1].drop(columns=["gender"])

In [ ]:
Po_data=Po_data[Po_data["count"]<=(Po_data["count"].describe()["75%"]+Po_data["count"].describe()["std"])]
Po_data=Po_data.groupby("date", as_index=False).apply(count_female_male).droplevel(0)
Po_data

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
sns.heatmap(Po_data.corr(), annot=True, ax=ax)

#### build polynomial regression model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.utils import shuffle

In [ ]:
Target_data=Po_data.loc[:,['Female']].values
Feature_data=Po_data.loc[:,["dew_point"]] .values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split( Feature_data , Target_data , test_size = 0.3, random_state = 0)

In [ ]:
def compute_mse_cv(X, Y, n_degree=11):
    mse = []
    std = []
    for degree in range(1,n_degree):
        poly_features = PolynomialFeatures(degree=degree)
        sklreg = linear_model.LinearRegression()
        pipeline = Pipeline([("polynomial_features", poly_features),
                             ("linear_regression", sklreg)])
        pipeline.fit(X, Y)

        # Evaluate the models using crossvalidation
        scores = cross_val_score(pipeline, X, Y,
                                 scoring="neg_mean_squared_error", 
                                 cv=10, error_score=np.nan)
        
        mse.append(-scores.mean())
        std.append(scores.std())       
    return mse,std

In [ ]:
def compute_mse(X_train, Y_train, X_test, Y_test, n_degree=11):
    train_mse = []
    test_mse = []
    for degree in range(1,n_degree):
        poly_features = PolynomialFeatures(degree=degree)
        sklreg = linear_model.LinearRegression()
        pipeline = Pipeline([("polynomial_features", poly_features),
                             ("linear_regression", sklreg)])
        pipeline.fit(X_train, Y_train)
        Y_train_pred = pipeline.predict(X_train)
        Y_test_pred = pipeline.predict(X_test)

        train_mse.append(mean_squared_error(Y_train, Y_train_pred))
        test_mse.append(mean_squared_error(Y_test, Y_test_pred))
    return train_mse,test_mse

def compute_scale_cov(X_train, Y_train, X_test, Y_test,scale_model, n_degree=11):
    train_mse = []
    test_mse = []
    for degree in range(1,n_degree):
        poly_features = PolynomialFeatures(degree=degree)
        sklreg = linear_model.LinearRegression()
        pipeline = Pipeline([("scaler",scale_model),
                             ("polynomial_features", poly_features),
                             ("linear_regression", sklreg)])
        pipeline.fit(X_train, Y_train)
        Y_train_pred = pipeline.predict(X_train)
        Y_test_pred = pipeline.predict(X_test)

        train_mse.append(mean_squared_error(Y_train, Y_train_pred))
        test_mse.append(mean_squared_error(Y_test, Y_test_pred))
    return train_mse,test_mse

In [ ]:
mse,std = compute_mse_cv(X_train,Y_train,10)
plt.plot(range(1,len(mse)+1),mse, label='MSE')
plt.plot(range(1,len(std)+1),std, label='std')
plt.legend();

In [ ]:
train_mse,test_mse = compute_mse(X_train,Y_train,X_test,Y_test,15)
plt.plot(range(1,len(train_mse)+1),train_mse, label='train')
plt.plot(range(1,len(test_mse)+1),test_mse, label='test')
plt.title('MSE')
plt.legend();

Performances in normalization and standardization

In [ ]:
stand_scaler= sk.preprocessing.StandardScaler()
norm_scaler=sk.preprocessing.MinMaxScaler()
normalize_scaler=sk.preprocessing.Normalizer()

In [ ]:
train_mse,test_mse = compute_scale_cov(X_train,Y_train,X_test,Y_test,norm_scaler,15)
plt.plot(range(1,len(train_mse)+1),train_mse, label='train')
plt.plot(range(1,len(test_mse)+1),test_mse, label='test')
plt.title('MSE')
plt.legend();

choose 13 degree as the best degree

In [ ]:
poly_features = PolynomialFeatures(degree=4)
sklreg = linear_model.LinearRegression()
polpipeline = Pipeline([("polynomial_features", poly_features),
                     ("linear_regression", sklreg)])
polpipeline.fit(X_train,Y_train)
Predict_Y=polpipeline.predict(X_test)

In [ ]:
results_df = pd.DataFrame()
soclling=["neg_mean_absolute_error","neg_mean_squared_error","neg_root_mean_squared_error","r2"]
series=cross_val(X_train,Y_train,polpipeline,soclling)
series.name="Polynomial model"
results_df=results_df.append(series)
results_df

In [ ]:
#test_pred=evaluate(Y_test,Predict_Y)
print('Test set evaluation:\n_____________________________________')
print_evaluate(Y_test, Predict_Y)